In [19]:
import pandas as pd
import numpy as np
import xgboost as xgb

import warnings
warnings.simplefilter("ignore")

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

In [20]:
datapath = './data/heart.csv'
data = pd.read_csv(datapath)
data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [21]:
X_columns_drop = ['target', 'cp']


X = data.drop(X_columns_drop, axis=1)
y = data['target']


X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=42)

In [22]:
def xgbooster(X, y):
    params = {'early_stopping_rounds':3, 'eval_set':[(X_test, y_test)], 'verbose':False}
    clf_xgb = xgb.XGBClassifier(n_estimators = 700, learning_rate = 0.0025, n_jobs = 12, use_label_encoder=False, verbosity=0)
    scores = cross_val_score(clf_xgb, X, y, cv=5, scoring='f1', fit_params=(params))
    print("Accuracy scores: ", ((scores.mean())*100), "%")


In [23]:
xgbooster(X, y)

Accuracy scores:  82.18822530166348 %


In [24]:
clf_xgb.fit(X_train, y_train, early_stopping_rounds=3, eval_set=[(X_test, y_test)], verbose=False)
prediction = clf_xgb.predict(X_test)
print("Accuracy Score: " + str(accuracy_score(y_test, prediction)))

Accuracy Score: 0.7540983606557377


In [25]:
results = clf_xgb.evals_result()
# print(results)

In [26]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaled_X = scaler.fit_transform(X)
scaled_X = pd.DataFrame(scaled_X, columns=X.columns)
scaled_X.head()

,age,sex,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,0.708333,1.0,0.481132,0.244292,1.0,0.0,0.603053,0.0,0.370968,0.0,0.0,0.333333
1,0.166667,1.0,0.339623,0.283105,0.0,0.5,0.885496,0.0,0.564516,0.0,0.0,0.666667
2,0.250000,0.0,0.339623,0.178082,0.0,0.0,0.770992,0.0,0.225806,1.0,0.0,0.666667
3,0.562500,1.0,0.245283,0.251142,0.0,0.5,0.816794,0.0,0.129032,1.0,0.0,0.666667
4,0.583333,0.0,0.245283,0.520548,0.0,0.5,0.702290,1.0,0.096774,1.0,0.0,0.666667


In [27]:
xgbooster(scaled_X, y)

Accuracy scores:  82.35592974334051 %
